# Build an inference script to run the Keras mask classifier built in Teachable Machine

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp -r /content/gdrive/MyDrive/PGSS2023/PGSS2023--ClassDrive/Week03/kerasTeachableMachineModel/ /content/

In [22]:
!cp -r /content/gdrive/MyDrive/PGSS2023/MaskData/with_mask/101-with-mask.jpg .

In [21]:
!cp /content/gdrive/MyDrive/PGSS2023/MaskData/without_mask/105.jpg .

In [23]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/kerasTeachableMachineModel/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/kerasTeachableMachineModel/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
# image = Image.open("/content/101-with-mask.jpg").convert("RGB")
image = Image.open("/content/105.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


1/1 [==============================] - 1s 838ms/step
Class: NoMask
Confidence Score: 1.0


In [24]:
# Explore the model to see what its architecture is

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_5 (Sequential)   (None, 2)                 128300    
                                                                 
Total params: 538,508
Trainable params: 524,428
Non-trainable params: 14,080
_________________________________________________________________


# Develop an inference SCRIPT that can run this model as an encapsulated FUNCTION

In [25]:
%%writefile runinference_keras_maskClassifier.py
# Here's the code refactored into a class with methods for loading the model and making predictions,
# and a main function that creates an instance of the class and calls its methods:

import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps


class KerasImageClassifier:
    def __init__(self, model_path, labels_path, image_path):
        # Disable scientific notation for clarity
        np.set_printoptions(suppress=True)

        self.model_path = model_path
        self.labels_path = labels_path
        self.image_path = image_path

        self.model = None
        self.class_names = None
        self.data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    def load_model_and_labels(self):
        # Load the model
        self.model = load_model(self.model_path, compile=False)
        # Load the labels
        self.class_names = open(self.labels_path, "r").readlines()

    def predict_image_class(self):
        # Open the image file
        image = Image.open(self.image_path).convert("RGB")

        # Resizing the image to be at least 224x224 and then cropping from the center
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

        # Turn the image into a numpy array
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Load the image into the array
        self.data[0] = normalized_image_array

        # Predicts the model
        prediction = self.model.predict(self.data)
        index = np.argmax(prediction)
        class_name = self.class_names[index]
        confidence_score = prediction[0][index]

        return class_name, confidence_score


"""
def main():
    model_path = "kerasTeachableMachineModel/keras_model.h5"
    labels_path = "kerasTeachableMachineModel/labels.txt"
    image_path = "105.jpg"

    classifier = KerasImageClassifier(model_path, labels_path, image_path)
    classifier.load_model_and_labels()
    class_name, confidence_score = classifier.predict_image_class()

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", confidence_score)
"""

import argparse

def main():
    # Create argument parser
    parser = argparse.ArgumentParser(description='Image Classifier')
    parser.add_argument('image', help='Path to the image that needs to be processed')

    # Parse arguments
    args = parser.parse_args()

    model_path = "kerasTeachableMachineModel/keras_model.h5"
    labels_path = "kerasTeachableMachineModel/labels.txt"
    image_path = args.image

    classifier = KerasImageClassifier(model_path, labels_path, image_path)
    classifier.load_model_and_labels()
    class_name, confidence_score = classifier.predict_image_class()

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", confidence_score)


if __name__ == "__main__":
    main()



Writing runinference_keras_maskClassifier.py


## Call the script that we just wrote and see what its results are when called on the command line

In [26]:
! python /content/runinference_keras_maskClassifier.py "105.jpg"

2023-07-28 19:44:15.726674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 2s 2s/step
Class: NoMask
Confidence Score: 1.0


In [27]:
! python /content/runinference_keras_maskClassifier.py "/content/101-with-mask.jpg"

2023-07-28 19:44:27.029219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 1s 897ms/step
Class: Mask
Confidence Score: 1.0


# Work out what the requirements of the colab machine are to run this model

In [28]:
! pip freeze > requirements.txt

In [29]:
!pip install pipreqs

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=00b20003e68dcae5336122e5f333211bdb88df348d26a2172be8974e111f7b4c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [30]:
! mkdir /content/CleanCode

In [33]:
! cp /content/runinference_keras_maskClassifier.py  /content/CleanCode

In [34]:
! pipreqs /content/CleanCode

Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /content/CleanCode/requirements.txt


# Build a Gradio app to expose the trained image based classification model

In [35]:
from google.colab import output
output.serve_kernel_port_as_window(7860)

<IPython.core.display.Javascript object>

In [36]:
%%writefile gradioApp.py
import gradio as gr
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("kerasTeachableMachineModel/keras_model.h5", compile=False)

# Load the labels
class_names = open("kerasTeachableMachineModel/labels.txt", "r").readlines()


def predict_image_class(image: np.ndarray, submit: bool):
    if submit:
        # Save the image to disk
        Image.fromarray(image).save("saved_image.jpg")

        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

        # Ensure the image is RGB format
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)

        # Convert image to a numpy array
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Load the image into the array
        data[0] = normalized_image_array

        # Run prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        return f"Class: {class_name}\nConfidence Score: {confidence_score}"
    else:
        return "Submit the image to get the prediction."


iface = gr.Interface(
    fn=predict_image_class,
    inputs=[gr.inputs.Image(shape=(224,224), source="webcam"), gr.inputs.Checkbox(label="Submit")],
    outputs="text"
)

iface.launch()


Writing gradioApp.py


In [37]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [38]:
!python gradioApp.py

2023-07-28 19:46:16.123763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/gradioApp.py:48: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Image(shape=(224,224), source="webcam"), gr.inputs.Checkbox(label="Submit")],
/content/gradioApp.py:48: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Image(shape=(224,224), source="webcam"), gr.inputs.Checkbox(label="Submit")],
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2133, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occ